Author: **Jaydeep Godbole** \\
Roll Number: **17IE10039**

In [1]:
from google.colab import drive
drive.mount('drive')

Mounted at drive


In [2]:
!pip install git+git://github.com/andreinechaev/nvcc4jupyter.git

  Cloning git://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-kg6txps4
  Running command git clone -q git://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-kg6txps4
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-cp37-none-any.whl size=4307 sha256=994ac4a1a928813d2c7f0c066f8319477cd3795c8f8798fedeac54c0cab0e17b
  Stored in directory: /tmp/pip-ephem-wheel-cache-tka8zdsf/wheels/10/c2/05/ca241da37bff77d60d31a9174f988109c61ba989e4d4650516
Successfully built NVCCPlugin


In [3]:
%load_ext nvcc_plugin

created output directory at /content/src
Out bin /content/result.out


In [4]:
%%cuda --name utils.h
#ifndef UTILS
#define UTILS

#include <vector>
#include <cstdlib>
namespace utils{
class graph
{
 public:
 int n;
 int m;
 int * edges_from;
 int * edges_to;
 int * edges;
 int * scores;

 void print_edges()
 {
  for(int i = 0; i < n; i++)
  {
   std::cout << "Starting " << i;
   /*for(int j = edges_from[i]; j <= edges_to[i]; j++)
   {
    std::cout << edges[j] << std::endl;
   }*/
   if (edges_to[i] == -1)
   {
    std::cout << 0 << std::endl;
   }
   else
   {
    std::cout << (edges_to[i]-edges_from[i] + 1) << std::endl;
   }    
    
  }
 }
 void print_scores_to_file(char * file);
};

graph initialize(std::vector<std::vector<int>> connections)
{
 utils::graph g;
 g.m = connections.size(); // Number of edges
 g.n = 0;
 for(int i = 0; i < g.m; i++)
 {
  if(connections[i][0]>g.n) g.n = connections[i][0];
  if(connections[i][1]>g.n) g.n = connections[i][1];
  
 }
 g.n += 1; // Number of nodes = max_idx + 1

 g.edges_from = new int[g.n]; //edges_from[i] is the index which neighbors of node i start. go up till edges_to[i]
 g.edges_to = new int[g.n]; 
 g.edges = new int[2*g.m];
 g.scores = new int[g.n];

 for(int i = 0; i < g.n; i++)
 {
  g.edges_from[i] = -1;
  g.edges_to[i] = -1;
 }

 int curr_edge_pointer = 0;

 /* Changing here               ------------
 for(int i = 0; i < g.n; i++)
 {
  bool isassigned = 0;
  for(int j = 0; j < g.m; j++)
  {
   if(connections[j][0]==i)
   {
    if(!isassigned)
    {
     g.edges_from[i] = curr_edge_pointer;
     isassigned = 1;
    }
    g.edges[curr_edge_pointer] = connections[j][1];
    curr_edge_pointer += 1; 
   }
   else if(connections[j][1]==i)
   {
    if(!isassigned)
    {
     g.edges_from[i] = curr_edge_pointer;
     isassigned = 1;
    }
    g.edges[curr_edge_pointer] = connections[j][0];
    curr_edge_pointer += 1; 
   }
   if(isassigned)
   {
   g.edges_to[i] = curr_edge_pointer - 1;
   }
  }
 }Till here  -----------------------
*/ 
 //  New -------------------
 for(int i = 0; i < g.n; i++)
 {
  for(int j = 0; j < g.m; j++)
  {
   if(connections[j][0]==i || connections[j][1]==i)
   {
    if (g.edges_from[i] == -1)
    {
     g.edges_from[i] = curr_edge_pointer;
    }
    if(connections[j][0]==i)    g.edges[curr_edge_pointer] = connections[j][1];
    else g.edges[curr_edge_pointer] = connections[j][0];
    curr_edge_pointer += 1;
   }
  }
  if(g.edges_from[i] != -1)
  {
   g.edges_to[i] = curr_edge_pointer-1;
  }
 }
 // New over ----------------------

 std::cout << g.n << " " << g.m << std::endl;
 std::cout << curr_edge_pointer/2 << std::endl;

 return g;
}

}
#endif

'File written in /content/src/utils.h'

In [7]:
%%cuda --name work_efficient_kernels.h
#ifndef KERNELS
#define KERNELS

#include </content/src/utils.h>
#include <cuda.h>
#include <cuda_runtime.h>
#include <vector>
#include <cstdlib>

__global__ void get_bc(const int * edges_from, const int * edges_to, const int * edges, const int m, const int n, float * bc, int * Q1, int * Q2, int * Q1_len, int * Q2_len, int * dist, int * num_shortest_paths, int * S, int * S_ends, int * S_len, float * bc_local)
{
 int i = blockIdx.x*blockDim.x + threadIdx.x;
 // Initialize all global arrays and stuff to 0 

 for(int k = 0; k < (n + gridDim.x*blockDim.x - 1)/(gridDim.x*blockDim.x); k++)
 {
  if(k*gridDim.x*blockDim.x + i < n)
  {
   if(k==0 && i==0)
   {
    Q1_len[0] = 0;
    Q2_len[0] = 0;  // Empty queues
   }
   bc[i + k*gridDim.x*blockDim.x] = 0;
   bc_local[i + k*gridDim.x*blockDim.x] = 0;
   Q1[i + k*gridDim.x*blockDim.x] = 0;
   Q2[i + k*gridDim.x*blockDim.x] = 0;
   dist[i + k*gridDim.x*blockDim.x] = 0;
   num_shortest_paths[i + k*gridDim.x*blockDim.x] = 0;
   S[i + k*gridDim.x*blockDim.x] = 0;
   S_ends[i + k*gridDim.x*blockDim.x] = 0;
  }
 }

 __syncthreads();
 

 for(int source = 0; source < n; source++)
 {
  __syncthreads();
  // New node, so set dist, num_shortest paths and bc_local to be zero!
  
  for(int k = 0; k < (n + gridDim.x*blockDim.x - 1)/(gridDim.x*blockDim.x); k++)
  {
    if(k*gridDim.x*blockDim.x + i < n)
    {
      if(k==0 && i==0)
      {
      S_len[0] = 0;
      Q1_len[0] = 0;
      Q2_len[0] = 0;
      }
      bc_local[i + k*gridDim.x*blockDim.x] = 0;
      Q1[i + k*gridDim.x*blockDim.x] = 0;
      Q2[i + k*gridDim.x*blockDim.x] = 0;
      dist[i + k*gridDim.x*blockDim.x] = INT_MAX;
      num_shortest_paths[i + k*gridDim.x*blockDim.x] = 0;
      S[i + k*gridDim.x*blockDim.x] = 0;
      S_ends[i + k*gridDim.x*blockDim.x] = 0;
      }
  }

  __syncthreads();

  int max_penultimate_depth = 0;

  // Run the shortest path calculation, from the source node outwards
  
  if(i==0)
  {
   // Only setting the source node here
   Q1[0] = source;
   Q1_len[0] = 1;
   dist[source] = 0;
   num_shortest_paths[source] = 1;
   S[0] = source;
   S_ends[0] = 0;
   S_len[0] = 1;
  }
  // Now, Q1 has the source node, Q2 is empty, S, S_ends have been assigned

  int curr_depth = 0; // Depth is defined as the distance of the frontier from the source

  __syncthreads();

  while(1)
  {
   // There are still unexplored nodes. Everything till current depth is done
   // Find next frontier nodes in parallel

   for(int k = 0; k < (Q1_len[0] + gridDim.x*blockDim.x - 1)/(gridDim.x*blockDim.x); k++)
   {
    if(k*gridDim.x*blockDim.x + i < Q1_len[0])
    {
     for(int neighbors_idx = edges_from[Q1[k*gridDim.x*blockDim.x + i]]; neighbors_idx <= edges_to[Q1[k*gridDim.x*blockDim.x + i]]; neighbors_idx++)
     {
      // neighbors_idx is a neighbor of a node in the current frontier
      if(atomicCAS(&dist[edges[neighbors_idx]], INT_MAX, curr_depth + 1) == INT_MAX)
      {
       // It is an unexplored node
       int temp = atomicAdd(&Q2_len[0], 1);
       Q2[temp] = edges[neighbors_idx];
      }
      if(dist[edges[neighbors_idx]] == curr_depth + 1)
      {
       atomicAdd(&num_shortest_paths[edges[neighbors_idx]], num_shortest_paths[Q1[k*gridDim.x*blockDim.x + i]]);
      }
     }
    }
   }
   
   __syncthreads();

   int next_frontier_len = Q2_len[0];

   if(next_frontier_len == 0)
   {
    max_penultimate_depth = curr_depth - 1;
    break;
   }
   else
   {
    for(int k = 0; k < (next_frontier_len + gridDim.x*blockDim.x - 1)/(gridDim.x*blockDim.x); k++)
    {
     if(k*gridDim.x*blockDim.x + i < next_frontier_len)
     {
      // Move Q2 into Q1
      Q1[k*gridDim.x*blockDim.x + i] = Q2[k*gridDim.x*blockDim.x + i];
      S[k*gridDim.x*blockDim.x + i + S_len[0]] = Q2[k*gridDim.x*blockDim.x + i];
     }
    }
    __syncthreads();

    curr_depth = curr_depth + 1;
    S_ends[curr_depth] = S_ends[curr_depth - 1] + next_frontier_len;
    Q1_len[0] = next_frontier_len;
    S_len[0] = S_len[0] + next_frontier_len;
    Q2_len[0] = 0;
    __syncthreads();
   }
   __syncthreads();

  }
  
  while(max_penultimate_depth > 0)
  {
   for(int k = 0; k < (S_ends[max_penultimate_depth] - S_ends[max_penultimate_depth - 1] + gridDim.x*blockDim.x - 1)/(gridDim.x*blockDim.x); k++)
   {
    if(k*gridDim.x*blockDim.x + i < S_ends[max_penultimate_depth] - S_ends[max_penultimate_depth - 1])
    {
     int curr_frontier_node = S[S_ends[max_penultimate_depth - 1] + 1 + k*gridDim.x*blockDim.x + i];
     float curr_frontier_node_dependency = 0;
     int curr_frontier_node_paths = num_shortest_paths[curr_frontier_node];
     
     for(int neighbors_idx = edges_from[curr_frontier_node]; neighbors_idx <= edges_to[curr_frontier_node]; neighbors_idx++)
     {
      if(dist[edges[neighbors_idx]] == max_penultimate_depth + 1)
      {
       curr_frontier_node_dependency = curr_frontier_node_dependency + ((float)curr_frontier_node_paths)/(num_shortest_paths[edges[neighbors_idx]]) * (1.0 + bc_local[edges[neighbors_idx]]);
      }
     }
     bc_local[curr_frontier_node] = curr_frontier_node_dependency;
    }
   }

   __syncthreads();
   max_penultimate_depth = max_penultimate_depth - 1;
  }

  __syncthreads();

  // Dependency d_s(v) has been stored in bc_local, and so this will be parallelly accumulated in bc

  for(int k = 0; k < (n + gridDim.x*blockDim.x - 1)/(gridDim.x*blockDim.x); k++)
  {
   if(k*gridDim.x*blockDim.x + i < n)
   {
    bc[k*gridDim.x*blockDim.x + i] = bc[k*gridDim.x*blockDim.x + i] + bc_local[k*gridDim.x*blockDim.x + i];
   }
  }
  __syncthreads();
 }

}

float * work_efficient_bc_gpu(utils::graph g)
{
 cudaDeviceProp prop;
 cudaError_t err = cudaSuccess;
 err = cudaGetDeviceProperties(&prop, 0);
 if(err != cudaSuccess)
 {
  std::cout << "Failed" << std::endl;
  exit(EXIT_FAILURE);
 }
 std::cout << "Chosen Device: " << prop.name << std::endl;
 std::cout << "Compute Capability: " << prop.major << "." << prop.minor << std::endl;
 std::cout << "Number of Streaming Multiprocessors: " << prop.multiProcessorCount << std::endl;
 std::cout << "Size of Global Memory: " << prop.totalGlobalMem/(float)(1024*1024*1024) << " GB"<< std::endl;

 int max_threads_per_block = prop.maxThreadsPerBlock;
 int num_SMs = prop.multiProcessorCount;
 
 int * d_edges = NULL;
 int * d_edges_from = NULL;
 int * d_edges_to = NULL;
 float * d_bc = NULL;
 float * d_bc_local = NULL;
 int * Q1 = NULL;
 int * Q2 = NULL;
 int * Q1_len = NULL;
 int * Q2_len = NULL;
 int * S_len = NULL;
 int * dist = NULL;
 int * num_shortest_paths = NULL;
 int * S = NULL;
 int * S_ends = NULL;


 size_t size = g.n*sizeof(int);
 err = cudaMalloc((void**)&d_edges_from, size);
 if(err!=cudaSuccess){std::cout<<"1"<<cudaGetErrorString(err)<<std::endl; exit(EXIT_FAILURE);}
 err = cudaMemcpy(d_edges_from, g.edges_from, size, cudaMemcpyHostToDevice);
 if(err!=cudaSuccess){std::cout<<cudaGetErrorString(err)<<std::endl; exit(EXIT_FAILURE);}
 err = cudaMalloc((void**)&d_edges_to, size);
 if(err!=cudaSuccess){std::cout<<cudaGetErrorString(err)<<std::endl; exit(EXIT_FAILURE);}
 err = cudaMemcpy(d_edges_to, g.edges_to, size, cudaMemcpyHostToDevice);
 if(err!=cudaSuccess){std::cout<<cudaGetErrorString(err)<<std::endl; exit(EXIT_FAILURE);}
 err = cudaMalloc((void**)&Q1, size);
 if(err!=cudaSuccess){std::cout<<cudaGetErrorString(err)<<std::endl; exit(EXIT_FAILURE);}
 err = cudaMalloc((void**)&Q2, size);
 if(err!=cudaSuccess){std::cout<<cudaGetErrorString(err)<<std::endl; exit(EXIT_FAILURE);}
 err = cudaMalloc((void**)&dist, size);
 if(err!=cudaSuccess){std::cout<<cudaGetErrorString(err)<<std::endl; exit(EXIT_FAILURE);}
 err = cudaMalloc((void**)&num_shortest_paths, size);
 if(err!=cudaSuccess){std::cout<<cudaGetErrorString(err)<<std::endl; exit(EXIT_FAILURE);}
 err = cudaMalloc((void**)&S, size);
 if(err!=cudaSuccess){std::cout<<cudaGetErrorString(err)<<std::endl; exit(EXIT_FAILURE);}
 err = cudaMalloc((void**)&S_ends, size);
 if(err!=cudaSuccess){std::cout<<cudaGetErrorString(err)<<std::endl; exit(EXIT_FAILURE);}
 
 size = sizeof(int);
 err = cudaMalloc((void**)&Q1_len, size);
 if(err!=cudaSuccess){std::cout<<cudaGetErrorString(err)<<std::endl; exit(EXIT_FAILURE);}
 err = cudaMalloc((void**)&Q2_len, size);
 if(err!=cudaSuccess){std::cout<<cudaGetErrorString(err)<<std::endl; exit(EXIT_FAILURE);}
 err = cudaMalloc((void**)&S_len, size);
 if(err!=cudaSuccess){std::cout<<cudaGetErrorString(err)<<std::endl; exit(EXIT_FAILURE);}
 
 size = g.n*sizeof(float);
 err = cudaMalloc((void**)&d_bc, size);
 if(err!=cudaSuccess){std::cout<<cudaGetErrorString(err)<<std::endl; exit(EXIT_FAILURE);}
 err = cudaMalloc((void**)&d_bc_local, size);
 if(err!=cudaSuccess){std::cout<<cudaGetErrorString(err)<<std::endl; exit(EXIT_FAILURE);}
 
 size = 2*g.m*sizeof(int);
 err = cudaMalloc((void**)&d_edges, size);
 if(err!=cudaSuccess){std::cout<<cudaGetErrorString(err)<<std::endl; exit(EXIT_FAILURE);}
 err = cudaMemcpy(d_edges, g.edges, size, cudaMemcpyHostToDevice);
 if(err!=cudaSuccess){std::cout<<cudaGetErrorString(err)<<std::endl; exit(EXIT_FAILURE);}
 
 cudaEvent_t start, stop;
 cudaEventCreate(&start);
 cudaEventCreate(&stop);

 cudaEventRecord(start);
 get_bc<<<num_SMs, max_threads_per_block>>>(d_edges_from, d_edges_to, d_edges, g.m, g.n, d_bc, Q1, Q2, Q1_len, Q2_len, dist, num_shortest_paths, S, S_ends, S_len, d_bc_local);
 cudaEventRecord(stop);
 cudaEventSynchronize(stop);
 float milliseconds = 0;
 cudaEventElapsedTime(&milliseconds, start, stop);
 
 std::cout << "The running time is " << milliseconds << "milliseconds" << std::endl;
 
 float * bc_calculated = new float[g.n];

 for(int i = 0; i < g.n; i++)
 {
  bc_calculated[i] = 1.0;
 }

 size = g.n*sizeof(float);
 err = cudaMemcpy(bc_calculated, d_bc, size, cudaMemcpyDeviceToHost);
 if(err!=cudaSuccess){std::cout<<cudaGetErrorString(err)<<std::endl; exit(EXIT_FAILURE);}

 return bc_calculated;

}

#endif

'File written in /content/src/work_efficient_kernels.h'

In [9]:
%%cuda --name work_efficient_main.cu

#include <iostream>
#include <fstream>
#include <sstream>
#include <string>
#include <vector>
#include <cstdlib>
#include <boost/algorithm/string.hpp>
#include </content/src/utils.h>
#include </content/src/work_efficient_kernels.h>

using namespace std;
class graph
{
 // Defining the graph data structure for handling large graphs
 public:
 int n; // Number of nodes in the graph
 int m; // Number of edges in the graph
 //std::vector<int*> begin;
 //std::vector<int*> end;
 int * begin;
 int * end;
 int * edges;


};
int main(int argc, char** argv)
{
 // First argument is the graph name (relative to the main directory).

 std::string s = argv[1];

 if(s.find(".txt") != std::string::npos)
  {
		;//return 0;//generate_graph(file);
	}
	else if(s.find(".edge") != std::string::npos)
	{
		;//return 0;//generate_graph(file);
	}
	else
	{
		std::cerr << "Error: Unsupported file type." << std::endl;
		exit(-1);
	}
 std::string datapath = "drive/MyDrive/HP3/Betweenness_Centrality/dataset/"; 
 s = datapath + s;
 std::cout << s << std::endl;
 std::fstream fin(s);
 std::string line;
 std::fstream graph_file;
 graph_file.open(s, ios::in);
 if(!graph_file)
 {
  cout << "File doesn't exist" << endl;
 }
 else
 {
  cout << "Graph file exists" << endl;
 }
 vector<vector<int>> edges;
 vector<int> temp;
 while (getline(fin, line)) {
  temp.clear();
  // Split line into tab-separated parts
  std::vector<std::string> parts;
  boost::algorithm::split(parts, line, boost::is_any_of(" "));
  // TODO Your code goes here!
  for(int i = 0; i < parts.size(); i++)
  {
   stringstream s(parts[i]);
   int k;
   s >> k;
   temp.push_back(k);
  }
  if(temp.size()!=2){
      cout << temp.size();
      cerr << "Format incorrect" << endl;
      return -1;
  }
 edges.push_back(temp);
 }
 fin.close();
 cout << "Number of edges in the graph: " << edges.size() << endl;

 utils::graph g = utils::initialize(edges);

 float * bc = work_efficient_bc_gpu(g);

 /*for(int i = 0; i < g.n; i++)
 {
  g.scores[i] = bc[i];
  std::cout << bc[i] << std::endl;
 }*/
 
 return 0;

 
}

'File written in /content/src/work_efficient_main.cu'

In [10]:
!nvcc /content/src/work_efficient_main.cu -o /content/src/work_efficient_main

In [12]:
!/content/src/work_efficient_main CA-GrQc-new.txt

drive/MyDrive/HP3/Betweenness_Centrality/dataset/CA-GrQc-new.txt
Graph file exists
Number of edges in the graph: 28980
5242 28980
28974
Chosen Device: Tesla T4
Compute Capability: 7.5
Number of Streaming Multiprocessors: 40
Size of Global Memory: 14.7556 GB
The running time is 1431.65milliseconds
